
## 1일차 수업: 순환 신경망 (RNN), 장단기 기억망 (LSTM), 게이트 순환 유닛 (GRU) 심층 분석 및 파이토치 실습

오늘 수업에서는 기본적인 순환 신경망(RNN)부터 시작하여, 장기 의존성 문제를 해결하기 위해 등장한 LSTM, 그리고 LSTM의 간소화된 형태인 GRU까지 세 가지 순환 신경망 모델의 이론적 배경과 구조를 심층적으로 분석해 보겠습니다. 이론 학습 후에는 파이토치를 이용하여 각 모델을 간단하게 구현하고 텍스트 분류 작업을 수행하는 실습 시간을 갖도록 하겠습니다.

### 1. 순환 신경망 (RNN - Recurrent Neural Network)

**1.1 이론 복습:**

* RNN은 순차적인 데이터(sequential data)를 처리하기 위해 설계된 신경망으로, 이전 시점의 정보를 현재 시점의 입력과 함께 고려하여 문맥을 파악하는 데 강점을 가집니다.
* **핵심 아이디어:** 내부의 순환 구조를 통해 이전 시점의 은닉 상태(hidden state)를 현재 시점으로 전달하여 과거 정보를 기억하고 활용합니다.
* **수학적 표현 (간략화):**
    $\qquad h_t = \tanh(W_{ih} x_t + W_{hh} h_{t-1} + b_h)$
    $\qquad y_t = W_{ho} h_t + b_o$
    * $x_t$: 현재 시점의 입력
    * $h_t$: 현재 시점의 은닉 상태
    * $h_{t-1}$: 이전 시점의 은닉 상태
    * $y_t$: 현재 시점의 출력
    * $W_{ih}, W_{hh}, W_{ho}$: 학습 가능한 가중치 행렬
    * $b_h, b_o$: 학습 가능한 편향 벡터
    * $\tanh$: 활성화 함수

**1.2 주요 문제점:**

* **기울기 소실/폭주 (Vanishing/Exploding Gradient):** 긴 시퀀스 데이터를 처리할 때, 역전파 과정에서 기울기가 점차 작아지거나 커져 학습이 제대로 이루어지지 않는 문제입니다.
* **장기 의존성 문제 (Long-Term Dependency Problem):** 멀리 떨어진 단어 간의 관계를 학습하기 어렵습니다. 초기 시점의 정보가 뒤쪽 시점까지 제대로 전달되지 못하는 경향이 있습니다.

### 2. 장단기 기억망 (LSTM - Long Short-Term Memory)

**2.1 핵심 아이디어:**

* RNN의 장기 의존성 문제를 해결하기 위해 제안된 모델로, **게이트(gate)**라는 메커니즘을 도입하여 불필요한 정보를 잊고 필요한 정보를 기억하는 능력을 향상시켰습니다.
* **셀 상태 (Cell State):** 장기적인 정보를 저장하고 전달하는 역할을 하는 핵심적인 요소입니다. 은닉 상태와 별도로 존재하며, 게이트에 의해 제어됩니다.

**2.2 LSTM의 구조:**

* **입력 게이트 (Input Gate):** 현재 입력과 이전 은닉 상태를 기반으로 셀 상태에 저장할 새로운 정보를 결정합니다.
* **망각 게이트 (Forget Gate):** 이전 셀 상태에서 버릴 정보를 결정합니다.
* **출력 게이트 (Output Gate):** 현재 입력, 이전 은닉 상태, 그리고 업데이트된 셀 상태를 기반으로 현재 은닉 상태를 결정합니다.

**2.3 수학적 표현:**

$\qquad i_t = \sigma(W_{xi} x_t + W_{hi} h_{t-1} + b_i)$ (입력 게이트)
$\qquad f_t = \sigma(W_{xf} x_t + W_{hf} h_{t-1} + b_f)$ (망각 게이트)
$\qquad o_t = \sigma(W_{xo} x_t + W_{ho} h_{t-1} + b_o)$ (출력 게이트)
$\qquad \tilde{C}_t = \tanh(W_{xc} x_t + W_{hc} h_{t-1} + b_c)$ (새로운 셀 상태 후보)
$\qquad C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C}_t$ (현재 셀 상태)
$\qquad h_t = o_t \odot \tanh(C_t)$ (현재 은닉 상태)

* $\sigma$: 시그모이드 함수
* $\tanh$: 하이퍼볼릭 탄젠트 함수
* $\odot$: 요소별 곱셈

### 3. 게이트 순환 유닛 (GRU - Gated Recurrent Unit)

**3.1 핵심 아이디어:**

* LSTM의 복잡한 구조를 간소화하면서도 비슷한 성능을 내도록 설계된 모델입니다. 망각 게이트와 입력 게이트를 하나의 **업데이트 게이트 (Update Gate)**로 통합하고, 셀 상태와 은닉 상태를 합쳐 구조를 단순화했습니다.

**3.2 GRU의 구조:**

* **업데이트 게이트 (Update Gate):** 이전 은닉 상태를 얼마나 유지하고 새로운 입력을 얼마나 반영할지를 결정합니다. (LSTM의 망각 게이트와 입력 게이트의 역할을 통합)
* **리셋 게이트 (Reset Gate):** 이전 은닉 상태가 현재 상태 계산에 얼마나 영향을 미칠지를 결정합니다.

**3.3 수학적 표현:**

$\qquad z_t = \sigma(W_{xz} x_t + W_{hz} h_{t-1} + b_z)$ (업데이트 게이트)
$\qquad r_t = \sigma(W_{xr} x_t + W_{hr} h_{t-1} + b_r)$ (리셋 게이트)
$\qquad \tilde{h}_t = \tanh(W_{xh} x_t + W_{hh} (r_t \odot h_{t-1}) + b_h)$ (새로운 은닉 상태 후보)
$\qquad h_t = (1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t$ (현재 은닉 상태)



In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader as loader

In [136]:
# 간단한 예시 데이터 (단어와 레이블)
train_data = [
    ("this movie is great", 1),  # 1: positive
    ("the acting is bad", 0),   # 0: negative
    ("i enjoyed the film", 1),
    ("it was a terrible movie", 0),
]

vocab=set()
for sentence,_ in train_data:    
    for word in sentence.split():
        vocab.add(word)

vocab=sorted(list(vocab))
print(vocab)
vocab_size=len(vocab)
print(vocab_size)
word_to_index={word:i for i, word in enumerate(vocab)}
index_to_word={i: word for word, i in word_to_index.items()}


def sentence_to_indices(sentence):
    return [word_to_index[word]for word in sentence.split()]


#indexed_data=[(torch.tensor(sentence_to_indices(s)),torch.tensor([l]))for s,l in train_data]
indexed_data = [(torch.tensor(sentence_to_indices(s)).unsqueeze(0), torch.tensor([l]).float().unsqueeze(0)) for s, l in train_data]




['a', 'acting', 'bad', 'enjoyed', 'film', 'great', 'i', 'is', 'it', 'movie', 'terrible', 'the', 'this', 'was']
14


In [150]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, out_size):
        super(RNN,self).__init__()
        self.embbading=nn.Embedding(vocab_size,embedding_size)
        self.rnn=nn.RNN(embedding_size, hidden_size,batch_first=True)
        self.fc=nn.Linear(hidden_size, out_size)

    def forward(self,x):
        embedded=self.embbading(x)
        out, hidden=self.rnn(embedded)
        return torch.sigmoid(self.fc(hidden[-1]))



In [173]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, out_size):
        super(LSTM,self).__init__()
        self.embedding=nn.Embedding(vocab_size, embedding_size)
        self.lstm=nn.LSTM(embedding_size, hidden_size, batch_first=True)
        self.fc=nn.Linear(hidden_size, out_size)

    def forward(self, x):
        embedding=self.embedding(x)
        out, (hidden,cell)=self.lstm(embedding)
        return torch.sigmoid(self.fc(hidden[-1]))

In [185]:
class GRU(nn.Module):
    def __init__(self,vocab_size, embedding_size, hidden_size, out_size):
        super(GRU,self).__init__()
        self.embedding=nn.Embedding(vocab_size,embedding_size)
        self.gru=nn.GRU(embedding_size, hidden_size, batch_first=True)
        self.fc=nn.Linear(hidden_size, out_size)

    def forward(self, x):
        embedding=self.embedding(x)
        out,hidden=self.gru(embedding)
        return torch.sigmoid(self.fc(hidden[-1]))




In [187]:
device=torch.device('cuda' if torch.cuda.is_available()else'cpu')
print(device,"mode")
def train(model,num_epoch, train_data, optim, criter):
    model.train()
    total_loss=0.0
    for epoch in range(num_epoch):
        for data, target in train_data:
            model=model.to(device)
            data, target=data.to(device), target.to(device)
            optim.zero_grad()
            out=model(data)
            loss=criter(out, target.float())
            loss.backward()
            optim.step()
            total_loss+=loss.item()
        print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_data):.4f}')


cuda mode


In [189]:
# 하이퍼파라미터 설정
embedding_dim = 10
hidden_dim = 16
output_dim = 1  # 긍정/부정 (binary classification)
learning_rate = 0.01
num_epochs = 10

In [191]:

rnn_model=RNN(vocab_size, embedding_dim, hidden_dim, output_dim)
lstm_model=LSTM(vocab_size, embedding_dim, hidden_dim, output_dim)
gru_model=GRU(vocab_size, embedding_dim, hidden_dim, output_dim)

optimy_R=optim.Adam(rnn_model.parameters(),lr=learning_rate)
optimy_L=optim.Adam(lstm_model.parameters(),lr=learning_rate)
optimy_G=optim.Adam(gru_model.parameters(),lr=learning_rate)

criter=nn.BCELoss()

train(rnn_model, num_epochs, indexed_data, optimy_R, criter)
print("/////////////////////////////////////////////////////////////////")
train(lstm_model, num_epochs, indexed_data, optimy_L, criter)
print("/////////////////////////////////////////////////////////////////")
train(gru_model, num_epochs, indexed_data, optimy_G, criter)

Epoch 1, Loss: 0.7104
Epoch 2, Loss: 1.2524
Epoch 3, Loss: 1.6702
Epoch 4, Loss: 1.9707
Epoch 5, Loss: 2.1692
Epoch 6, Loss: 2.2949
Epoch 7, Loss: 2.3766
Epoch 8, Loss: 2.4324
Epoch 9, Loss: 2.4727
Epoch 10, Loss: 2.5032
/////////////////////////////////////////////////////////////////
Epoch 1, Loss: 0.7055
Epoch 2, Loss: 1.3511
Epoch 3, Loss: 1.9475
Epoch 4, Loss: 2.4814
Epoch 5, Loss: 2.9331
Epoch 6, Loss: 3.2858
Epoch 7, Loss: 3.5363
Epoch 8, Loss: 3.6981
Epoch 9, Loss: 3.7952
Epoch 10, Loss: 3.8519
/////////////////////////////////////////////////////////////////
Epoch 1, Loss: 0.7250
Epoch 2, Loss: 1.3704
Epoch 3, Loss: 1.9584
Epoch 4, Loss: 2.4821
Epoch 5, Loss: 2.9282
Epoch 6, Loss: 3.2841
Epoch 7, Loss: 3.5458
Epoch 8, Loss: 3.7231
Epoch 9, Loss: 3.8369
Epoch 10, Loss: 3.9093


 **은닉 상태($h_t$) 자체가 리셋 게이트($r_t$)와 업데이트 게이트($z_t$)의 역할을 수행하는 것은 아닙니다.**

**리셋 게이트($r_t$)와 업데이트 게이트($z_t$)는 별도의 계산 과정을 거쳐 생성되는 벡터입니다.** 이 게이트들은 **이전 은닉 상태($h_{t-1}$)와 현재 입력($x_t$)을 기반으로** 각각 어떤 정보를 버릴지 (리셋 게이트) 그리고 어떤 정보를 유지하고 새로운 정보를 얼마나 반영할지 (업데이트 게이트)를 결정하는 역할을 합니다.

**GRU의 정보 흐름을 다시 한번 정리해 드리겠습니다.**

1.  **업데이트 게이트($z_t$) 계산:** 이전 은닉 상태($h_{t-1}$)와 현재 입력($x_t$)을 가중치와 함께 시그모이드 함수에 통과시켜 0과 1 사이의 값을 갖는 벡터를 얻습니다. 이 값은 이전 은닉 상태를 얼마나 유지할지를 결정합니다.

2.  **리셋 게이트($r_t$) 계산:** 이전 은닉 상태($h_{t-1}$)와 현재 입력($x_t$)을 또 다른 가중치와 함께 시그모이드 함수에 통과시켜 0과 1 사이의 값을 갖는 벡터를 얻습니다. 이 값은 이전 은닉 상태가 새로운 후보 은닉 상태를 계산하는 데 얼마나 영향을 미칠지를 결정합니다.

3.  **새로운 은닉 상태 후보($\tilde{h}_t$) 계산:** 현재 입력($x_t$)과 **리셋 게이트($r_t$)와 요소별 곱셈($\odot$)을 거친 이전 은닉 상태($r_t \odot h_{t-1}$)** 를 함께 $\tanh$ 함수에 통과시켜 새로운 은닉 상태 후보를 생성합니다. 리셋 게이트가 0에 가까워지면 이전 은닉 상태의 영향이 거의 사라지고, 현재 입력에 더 집중하게 됩니다.

4.  **현재 은닉 상태($h_t$) 업데이트:** 업데이트 게이트($z_t$)를 사용하여 이전 은닉 상태($h_{t-1}$)와 새로운 은닉 상태 후보($\tilde{h}_t$)를 조합합니다.
    $\qquad h_t = (1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t$
    업데이트 게이트 값이 1에 가까우면 새로운 은닉 상태 후보의 정보가 많이 반영되고, 0에 가까우면 이전 은닉 상태의 정보가 많이 유지됩니다.

**결론적으로, 은닉 상태($h_t$)는 과거 정보를 저장하고 전달하는 역할을 수행하지만, 리셋 게이트($r_t$)와 업데이트 게이트($z_t$)는 이전 은닉 상태와 현재 입력을 기반으로 *별도로 계산되어* 정보의 흐름을 제어하는 역할을 합니다.** 이 게이트들의 조절을 통해 GRU는 장기 의존성을 효과적있다면 다시 편하게 질문해주세요!